In [79]:
# custom imports
from regression_class import RedditRegression as RR

# data saving imports
import pickle
import os

# time tracking
from datetime import datetime as dt

import pandas as pd

In [80]:
DOMAINS_INFILE = "../top10milliondomains.csv/top10milliondomains.csv"
domain_rank = pd.read_csv(DOMAINS_INFILE)

In [81]:
# PARAMS_DICT_INFILE = f"{OUTDIR}/input_params.p"
REGRESSION_INFILE = "regression_thread_data.p"
THREAD_INFILE = "clean_5_thread_data.p"

In [82]:
regdfs = pickle.load(open(REGRESSION_INFILE, 'rb'))
threaddfs = pickle.load(open(THREAD_INFILE, 'rb'))

In [84]:
domain_rank = domain_rank[['Domain', 'Open Page Rank']].rename(columns = {
    'Domain': 'domain',
    'Open Page Rank': 'open_page_rank'
})


In [85]:
domain_pagerank_dict = dict(zip(domain_rank.domain,domain_rank.open_page_rank))

In [86]:
def change_reddit_domains(domains_col):
    to_alter = domains_col[domains_col.str.contains('self.')]
    to_alter = to_alter.str.removeprefix('self.') + '.reddit.com'
    domains_col[domains_col.str.contains('self.')] = to_alter
    return domains_col

def get_pagerank_scores(domain_col):
    domain_col = change_reddit_domains(domain_col)
    new_domain_col = domain_col.map(domain_pagerank_dict, na_action=None).fillna(0)
    return new_domain_col


In [87]:
new_reg_dfs = {}
for key in threaddfs:
    posts = threaddfs[key][threaddfs[key].thread_id == threaddfs[key].id]
    posts['domain_pagerank'] = get_pagerank_scores(posts.domain)
    new_reg_dfs[key] = pd.merge(regdfs[key], posts[['thread_id', 'domain', 'domain_pagerank']], how='left', on='thread_id')

C:\Users\snuzz\AppData\Local\Temp\ipykernel_38356\2515556108.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  domains_col[domains_col.str.contains('self.')] = to_alter
C:\Users\snuzz\AppData\Local\Temp\ipykernel_38356\2515556108.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  domains_col[domains_col.str.contains('self.')] = to_alter
C:\Users\snuzz\AppData\Local\Temp\ipykernel_38356\950395547.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [89]:
pickle.dump(new_reg_dfs, open('regression_thread_data_april_2024.p', 'wb'))